# Alpha Vantage

In [65]:
import requests

api_key = 'SWJ8LKHNRYM6AP15'
ticker = 'TSLA'
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval=30min&apikey={api_key}'
r = requests.get(url)
data = r.json()

In [66]:
data

{'Meta Data': {'1. Information': 'Intraday (30min) open, high, low, close prices and volume',
  '2. Symbol': 'TSLA',
  '3. Last Refreshed': '2022-11-07 20:00:00',
  '4. Interval': '30min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (30min)': {'2022-11-07 20:00:00': {'1. open': '196.9600',
   '2. high': '197.4400',
   '3. low': '196.9600',
   '4. close': '197.3000',
   '5. volume': '130155'},
  '2022-11-07 19:30:00': {'1. open': '196.7300',
   '2. high': '196.9800',
   '3. low': '196.7200',
   '4. close': '196.9500',
   '5. volume': '66963'},
  '2022-11-07 19:00:00': {'1. open': '196.8500',
   '2. high': '196.8600',
   '3. low': '196.7000',
   '4. close': '196.7300',
   '5. volume': '64157'},
  '2022-11-07 18:30:00': {'1. open': '197.1500',
   '2. high': '197.1500',
   '3. low': '196.5000',
   '4. close': '196.8700',
   '5. volume': '120970'},
  '2022-11-07 18:00:00': {'1. open': '197.4501',
   '2. high': '197.6300',
   '3. low': '197.1000',
   '4. clo

# Scrape

In [59]:
import requests
from bs4 import BeautifulSoup

stocks = ['TSLA', 'MSFT', 'AAPL', 'GOOGL']
stock_data = []

def getData(stock):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
    url = f'https://finance.yahoo.com/quote/{stock}'
    
    r = requests.get(url, headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    data = {
        'symbol': stock,
        'price': soup.find('fin-streamer', {"class": "Fw(b) Fz(36px) Mb(-4px) D(ib)"}).text,
        'change': soup.find('fin-streamer', {"class": "Fw(500) Pstart(8px) Fz(24px)"}).text
    }
    
    return data

for tick in stocks:
    print(f"Getting stock data for: {tick}")
    stock_data.append(getData(tick))
    
print(stock_data)

Getting stock data for: TSLA
Getting stock data for: MSFT
Getting stock data for: AAPL
Getting stock data for: GOOGL
[{'symbol': 'TSLA', 'price': '191.34', 'change': '-5.74'}, {'symbol': 'MSFT', 'price': '228.47', 'change': '+0.60'}, {'symbol': 'AAPL', 'price': '139.26', 'change': '+0.34'}, {'symbol': 'GOOGL', 'price': '88.65', 'change': '+0.16'}]
